# Fast Agent - Build a SQL Agent fast!

<img src="./assets/LC_L1_top.png" align="left" width="500">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")  # check environmental variables
doublecheck_pkgs(pyproject_path="pyproject.toml", verbose=True)   # check packages

OPENAI_API_KEY=****here
LANGSMITH_API_KEY=****754b
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
Python 3.12.3 satisfies requires-python: >=3.11,<3.14
package                | required | installed | status | path                                                                           
---------------------- | -------- | --------- | ------ | -------------------------------------------------------------------------------
langgraph              | >=1.0.0  | 1.0.1     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain              | >=1.0.0  | 1.0.2     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain-core         | >=1.0.0  | 1.0.1     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain-openai       | >=1.0.0  | 1.0.1     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain-anthropic    

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

Define the runtime context to provide the agent and tools with access to the database.

In [3]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase


# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

<b>⚠️ Security Note:</b> This demo does not include a filter on LLM-generated commands. In production, you would want to limit the scope of LLM-generated commands. ⚠️   
This tool will connect to the database. Note the use of `get_runtime` to access the graph **runtime context**.

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

Add a system prompt to define your agents behavior.

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

Create your agent! Add a model, tools, a prompt, and the runtime access, and go!  You can choose many agents from our [integrations](https://docs.langchain.com/oss/python/integrations/providers) list. 

In [9]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="mistral:7b", temperature=0)

In [10]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

Here's a display of the agent ReAct Loop.

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

Run some queries. Notice:
- The agent does not have the database schema and will need to discover it independently.
- The agent may make mistakes! By returning error messages, the agent can self-correct its queries.
- Notice you invoke the agent with `agent.stream`.
    - This command and the `pretty_print` display the **messages** that communicate information between the model and the tools.
- Notice the agent doesn't remember the schema between invocations... More on this later!

In [11]:
question = "Which table has the largest number of entries?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which table has the largest number of entries?
================================== Ai Message ==================================

 To find out which table has the largest number of entries, I will first list all tables in the database and then count the rows for each one. Here's the step-by-step process:

1. List tables:
```sql
execute_sql({"query": "SELECT name FROM sqlite_master WHERE type='table';"})
```

2. Count rows for each table:

Assuming the list of tables is `[Table1, Table2, ...]`, I will create a SQL query to count the number of rows in each table and order them by the row count in descending order.

```sql
let table_counts = [];
for table in [Table1, Table2, ...]:
  execute_sql({"query": f"SELECT COUNT(*) as Rows FROM {table};"})
  .then(result => table_counts.push(result[0]))

execute_sql({
  "query": `WITH table_counts AS (
    SELECT '${Table1}' as Table, COUNT(*) as Rows FROM ${Table1}
  

In [ ]:
question = "Please list all of the tables"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "List the total entry for each table in the database"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**Create your own query here!**  Add some questions of your own.

In [ ]:
question = "TRY YOUR OWN QUERY HERE"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Let's try this Studio